In [1]:
import pandas as pd
import numpy as np
import datetime
import os
import gc
os.getcwd()

'/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update'

In [4]:
detailed_scores_df=pd.read_csv("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update/crm_newscore_20190107/" + 'dfrfm_wemail.csv',dtype=str)

detailed_scores_df['frmindex']=detailed_scores_df['frmindex'].apply(lambda x: str(int(float(x))).zfill(2))
detailed_scores_df['customer_zip_code']=detailed_scores_df['customer_zip_code'].apply(lambda x: x.zfill(5))
detailed_scores_df['frmindex']=detailed_scores_df['frmindex'].apply(lambda x:"D"+x)

detailed_scores_df.head(2)

,customer_id_hashed,frmindex,active,email_address_hash,customer_zip_code,zipcodegroup
0,3fab5ed9c18860c28e3eff24dfc8d02f4d9f417917b9d7...,D01,active,0978fbe2d87f554b240f791b291ebac6295679706fd628...,45692,P
1,982e2d92ca94e2ba3cb397a49dd73909553f532dcd84bb...,D01,active,21ced089d5370d961feaa053059cfe5559ac6ebfb09694...,28607,P


In [3]:
id_by_store_trans=pd.read_csv("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update/crm_newscore_20190107/id_by_store_based_on_2018_trans.csv",dtype=str,usecols=['customer_id_hashed','location_id'])

id_by_store_register=pd.read_csv("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update/crm_newscore_20190107/BL_id_by_register_store_JL_2019-01-18.csv",dtype=str)
id_by_store_register=id_by_store_register[['customer_id_hashed','sign_up_location']].rename(columns={"sign_up_location":"location_id"})

id_by_store=id_by_store_trans.append(id_by_store_register).drop_duplicates('customer_id_hashed')

del id_by_store_trans
del id_by_store_register
gc.collect()
id_by_store.shape

(25149873, 2)

In [5]:
Q3_Quadrant=pd.read_excel("/home/jian/Projects/Big_Lots/Analysis/2018_Q4/Planner_Request/zips_by_quadrant_20190111/Excel_BL_data_for_2018Q3_Quadrants_JL_2018-12-04_Q2Scale.xlsx",dtype=str,sheetname="2018_Q3_Quadrants")

Q3_Quadrant=Q3_Quadrant[['location_id','Quadrant in Graph 1-2']].rename(columns={"Quadrant in Graph 1-2":"Quadrant"}).drop_duplicates()
print(len(Q3_Quadrant['location_id'].unique()))
Q3_Quadrant.head(2)

id_by_store=pd.merge(id_by_store,Q3_Quadrant,on="location_id",how="left")
id_by_store['Quadrant']=id_by_store['Quadrant'].fillna("Quadrant X")
id_by_store=id_by_store[['customer_id_hashed','Quadrant']]

1363


,location_id,Quadrant
0,1,Quadrant IV
1,3,Quadrant II


In [14]:
detailed_scores_df=pd.merge(detailed_scores_df,id_by_store,on="customer_id_hashed",how="left")
detailed_scores_df['Quadrant']=detailed_scores_df['Quadrant'].fillna("Quadrant X")

In [23]:
detailed_scores_df['HML_Group']=np.where(detailed_scores_df['frmindex'].isin(['D01','D02','D03','D04']),"H",
                                        np.where(detailed_scores_df['frmindex'].isin(['D05','D06','D07']),"M","L"))

In [24]:
detailed_scores_df.to_csv("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update/crm_newscore_20190107/seg_details_store_with_X_quadrant_JL_"+str(datetime.datetime.now().date())+".csv",index=False)

In [27]:
summary=detailed_scores_df.groupby(['Quadrant','frmindex','active','zipcodegroup','HML_Group'])['email_address_hash'].count().to_frame().reset_index()
summary.shape


(210, 6)

In [28]:
summary.to_csv("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update/crm_newscore_20190107/summary_with_X_quadrant_JL_"+str(datetime.datetime.now().date())+".csv",index=False)

In [32]:
# Others removed, only 0-18 months included, and X replaced by III
detailed_scores_df=detailed_scores_df[detailed_scores_df['active']!="other"]
detailed_scores_df['Quadrant']=detailed_scores_df['Quadrant'].replace("Quadrant X","Quadrant III")
detailed_scores_df.shape

(18654857, 8)

In [35]:
detailed_scores_df['segment_decile']=detailed_scores_df['active']+"_"+detailed_scores_df['zipcodegroup']+"_"+detailed_scores_df['frmindex']

len(detailed_scores_df['segment_decile'].unique())


36

In [36]:
import random
random.seed(1)
total_rows=len(detailed_scores_df)

test_all_NoOthers_df=pd.DataFrame()
control_all_NoOthers_df=pd.DataFrame()

i_counter=0

for seg,group in detailed_scores_df.groupby(['segment_decile']):
    random_list=random.sample(range(len(group)), int(np.round(len(group)/total_rows*500000)))

    group=group.reset_index()
    del group['index']
    group=group.reset_index()
    df_control=group[group['index'].isin(random_list)]
    df_test=group[~group['index'].isin(random_list)]
    
    df_control['segment_decile']="C_"+df_control['segment_decile']
    df_test['segment_decile']="T_"+df_test['segment_decile']
    
    test_all_NoOthers_df=test_all_NoOthers_df.append(df_test)
    control_all_NoOthers_df=control_all_NoOthers_df.append(df_control)
    i_counter+=1
    print(i_counter,datetime.datetime.now())

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


1 2019-01-18 19:30:29.376495
2 2019-01-18 19:31:05.663022
3 2019-01-18 19:32:50.264307
4 2019-01-18 19:33:41.492508
5 2019-01-18 19:34:54.697206
6 2019-01-18 19:36:34.668640
7 2019-01-18 19:37:40.534564
8 2019-01-18 19:37:52.881357
9 2019-01-18 19:38:10.920252
10 2019-01-18 19:38:50.035193
11 2019-01-18 19:39:47.072993
12 2019-01-18 19:39:56.198768
13 2019-01-18 19:40:15.795402
14 2019-01-18 19:40:51.134995
15 2019-01-18 19:41:15.367027
16 2019-01-18 19:41:30.147655
17 2019-01-18 19:41:37.147788
18 2019-01-18 19:41:46.407548
19 2019-01-18 19:41:57.683902
20 2019-01-18 19:42:12.068319
21 2019-01-18 19:42:28.644721
22 2019-01-18 19:42:45.422409
23 2019-01-18 19:42:56.904865
24 2019-01-18 19:43:25.441906
25 2019-01-18 19:44:31.308831
26 2019-01-18 19:46:18.567905
27 2019-01-18 19:46:22.906090
28 2019-01-18 19:46:28.518829
29 2019-01-18 19:46:39.855380
30 2019-01-18 19:46:54.693931
31 2019-01-18 19:47:05.518813
32 2019-01-18 19:47:17.408162
33 2019-01-18 19:47:24.768932
34 2019-01-18 19:47

In [67]:
test_all_NoOthers_df['segment_2019Q1']="T_"+test_all_NoOthers_df['Quadrant']+"_"+test_all_NoOthers_df['zipcodegroup']+"_"+test_all_NoOthers_df['HML_Group']+"_2019Q1"
control_all_NoOthers_df['segment_2019Q1']="C_"+control_all_NoOthers_df['Quadrant']+"_"+control_all_NoOthers_df['zipcodegroup']+"_"+control_all_NoOthers_df['HML_Group']+"_2019Q1"


In [68]:
len(test_all_NoOthers_df['segment_2019Q1'].unique())

36

In [69]:
len(control_all_NoOthers_df['segment_2019Q1'].unique())

36

In [70]:
test_all_NoOthers_df['segment_2019Q1'].unique().tolist()

['T_Quadrant III_P_H_2019Q1',
 'T_Quadrant I_P_H_2019Q1',
 'T_Quadrant II_P_H_2019Q1',
 'T_Quadrant IV_P_H_2019Q1',
 'T_Quadrant II_P_M_2019Q1',
 'T_Quadrant III_P_M_2019Q1',
 'T_Quadrant I_P_M_2019Q1',
 'T_Quadrant IV_P_M_2019Q1',
 'T_Quadrant IV_P_L_2019Q1',
 'T_Quadrant I_P_L_2019Q1',
 'T_Quadrant II_P_L_2019Q1',
 'T_Quadrant III_P_L_2019Q1',
 'T_Quadrant I_S_H_2019Q1',
 'T_Quadrant IV_S_H_2019Q1',
 'T_Quadrant III_S_H_2019Q1',
 'T_Quadrant II_S_H_2019Q1',
 'T_Quadrant III_S_M_2019Q1',
 'T_Quadrant IV_S_M_2019Q1',
 'T_Quadrant I_S_M_2019Q1',
 'T_Quadrant II_S_M_2019Q1',
 'T_Quadrant I_S_L_2019Q1',
 'T_Quadrant II_S_L_2019Q1',
 'T_Quadrant IV_S_L_2019Q1',
 'T_Quadrant III_S_L_2019Q1',
 'T_Quadrant I_T_H_2019Q1',
 'T_Quadrant II_T_H_2019Q1',
 'T_Quadrant III_T_H_2019Q1',
 'T_Quadrant IV_T_H_2019Q1',
 'T_Quadrant IV_T_M_2019Q1',
 'T_Quadrant I_T_M_2019Q1',
 'T_Quadrant II_T_M_2019Q1',
 'T_Quadrant III_T_M_2019Q1',
 'T_Quadrant IV_T_L_2019Q1',
 'T_Quadrant I_T_L_2019Q1',
 'T_Quadrant II

In [71]:
control_all_NoOthers_df['segment_2019Q1'].unique().tolist()

['C_Quadrant IV_P_H_2019Q1',
 'C_Quadrant I_P_H_2019Q1',
 'C_Quadrant II_P_H_2019Q1',
 'C_Quadrant III_P_H_2019Q1',
 'C_Quadrant II_P_M_2019Q1',
 'C_Quadrant IV_P_M_2019Q1',
 'C_Quadrant III_P_M_2019Q1',
 'C_Quadrant I_P_M_2019Q1',
 'C_Quadrant I_P_L_2019Q1',
 'C_Quadrant IV_P_L_2019Q1',
 'C_Quadrant III_P_L_2019Q1',
 'C_Quadrant II_P_L_2019Q1',
 'C_Quadrant III_S_H_2019Q1',
 'C_Quadrant IV_S_H_2019Q1',
 'C_Quadrant I_S_H_2019Q1',
 'C_Quadrant II_S_H_2019Q1',
 'C_Quadrant IV_S_M_2019Q1',
 'C_Quadrant I_S_M_2019Q1',
 'C_Quadrant III_S_M_2019Q1',
 'C_Quadrant II_S_M_2019Q1',
 'C_Quadrant III_S_L_2019Q1',
 'C_Quadrant I_S_L_2019Q1',
 'C_Quadrant IV_S_L_2019Q1',
 'C_Quadrant II_S_L_2019Q1',
 'C_Quadrant IV_T_H_2019Q1',
 'C_Quadrant II_T_H_2019Q1',
 'C_Quadrant III_T_H_2019Q1',
 'C_Quadrant I_T_H_2019Q1',
 'C_Quadrant I_T_M_2019Q1',
 'C_Quadrant IV_T_M_2019Q1',
 'C_Quadrant III_T_M_2019Q1',
 'C_Quadrant II_T_M_2019Q1',
 'C_Quadrant I_T_L_2019Q1',
 'C_Quadrant IV_T_L_2019Q1',
 'C_Quadrant II

In [72]:
test_all_NoOthers_df.to_csv("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update/crm_newscore_20190107/all_test_no_others"+str(datetime.datetime.now())+".csv",index=False)
control_all_NoOthers_df.to_csv("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update/crm_newscore_20190107/all_control_no_others"+str(datetime.datetime.now())+".csv",index=False)


In [73]:
folder_write_inner = '/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update/crm_newscore_20190107/by_group_quadrant_24/'

i_counter=0
for seg_2019Q1,group in test_all_NoOthers_df.groupby('segment_2019Q1'):
    group=group[['customer_id_hashed','email_address_hash','segment_2019Q1']].rename(columns={"segment_2019Q1":"segment"})
    group.to_csv(folder_write_inner+seg_2019Q1+".csv",index=False)
    i_counter+=1
    print(i_counter,datetime.datetime.now(),seg_2019Q1,len(group))

1 2019-01-18 22:02:21.320227 T_Quadrant III_P_H_2019Q1 1213736
2 2019-01-18 22:02:46.976121 T_Quadrant III_P_L_2019Q1 542350
3 2019-01-18 22:03:33.435588 T_Quadrant III_P_M_2019Q1 1083738
4 2019-01-18 22:03:43.615634 T_Quadrant III_S_H_2019Q1 272264
5 2019-01-18 22:03:51.083657 T_Quadrant III_S_L_2019Q1 138387
6 2019-01-18 22:04:02.398096 T_Quadrant III_S_M_2019Q1 255081
7 2019-01-18 22:04:07.202981 T_Quadrant III_T_H_2019Q1 126424
8 2019-01-18 22:04:13.748779 T_Quadrant III_T_L_2019Q1 144495
9 2019-01-18 22:04:21.488696 T_Quadrant III_T_M_2019Q1 145909
10 2019-01-18 22:04:57.855368 T_Quadrant II_P_H_2019Q1 994184
11 2019-01-18 22:05:06.756308 T_Quadrant II_P_L_2019Q1 273657
12 2019-01-18 22:05:32.694091 T_Quadrant II_P_M_2019Q1 709274
13 2019-01-18 22:05:41.218035 T_Quadrant II_S_H_2019Q1 173513
14 2019-01-18 22:07:15.948587 T_Quadrant II_S_L_2019Q1 59974
15 2019-01-18 22:07:17.016481 T_Quadrant II_S_M_2019Q1 139476
16 2019-01-18 22:07:17.580643 T_Quadrant II_T_H_2019Q1 82589
17 2019-

In [74]:
i_counter=0
for seg_2019Q1,group in control_all_NoOthers_df.groupby('segment_2019Q1'):
    group=group[['customer_id_hashed','email_address_hash','segment_2019Q1']].rename(columns={"segment_2019Q1":"segment"})
    group.to_csv(folder_write_inner+seg_2019Q1+".csv",index=False)
    i_counter+=1
    print(i_counter,datetime.datetime.now(),seg_2019Q1,len(group))

1 2019-01-18 22:08:41.766445 C_Quadrant III_P_H_2019Q1 33498
2 2019-01-18 22:08:41.895213 C_Quadrant III_P_L_2019Q1 14936
3 2019-01-18 22:08:42.071154 C_Quadrant III_P_M_2019Q1 29807
4 2019-01-18 22:08:42.114665 C_Quadrant III_S_H_2019Q1 7591
5 2019-01-18 22:08:42.136751 C_Quadrant III_S_L_2019Q1 3807
6 2019-01-18 22:08:42.174068 C_Quadrant III_S_M_2019Q1 6964
7 2019-01-18 22:08:42.194902 C_Quadrant III_T_H_2019Q1 3452
8 2019-01-18 22:08:42.217242 C_Quadrant III_T_L_2019Q1 3937
9 2019-01-18 22:08:42.240377 C_Quadrant III_T_M_2019Q1 4069
10 2019-01-18 22:08:42.381352 C_Quadrant II_P_H_2019Q1 27224
11 2019-01-18 22:08:42.422715 C_Quadrant II_P_L_2019Q1 7418
12 2019-01-18 22:08:42.521943 C_Quadrant II_P_M_2019Q1 19665
13 2019-01-18 22:08:42.550912 C_Quadrant II_S_H_2019Q1 4861
14 2019-01-18 22:08:42.562164 C_Quadrant II_S_L_2019Q1 1624
15 2019-01-18 22:08:42.584726 C_Quadrant II_S_M_2019Q1 3859
16 2019-01-18 22:08:42.599827 C_Quadrant II_T_H_2019Q1 2333
17 2019-01-18 22:08:42.606524 C_Qua

In [75]:
summary_test_Q1=test_all_NoOthers_df.groupby(['segment_2019Q1','Quadrant','HML_Group','zipcodegroup'])['email_address_hash'].count().to_frame().reset_index().rename(columns={"email_address_hash":"id_count"})
summary_test_decile=test_all_NoOthers_df.groupby(['segment_decile','active','Quadrant','frmindex','zipcodegroup'])['email_address_hash'].count().to_frame().reset_index().rename(columns={"email_address_hash":"id_count","frmindex":"decile"})

summary_control_Q1=control_all_NoOthers_df.groupby(['segment_2019Q1','Quadrant','HML_Group','zipcodegroup'])['email_address_hash'].count().to_frame().reset_index().rename(columns={"email_address_hash":"id_count"})
summary_control_decile=control_all_NoOthers_df.groupby(['segment_decile','active','Quadrant','frmindex','zipcodegroup'])['email_address_hash'].count().to_frame().reset_index().rename(columns={"email_address_hash":"id_count","frmindex":"decile"})


In [76]:
writer=pd.ExcelWriter(folder_write_inner+"summary_count_without_18_months_plus_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
summary_test_Q1.to_excel(writer,"summary_test_Q1",index=False)
summary_test_decile.to_excel(writer,"summary_test_decile",index=False)

summary_control_Q1.to_excel(writer,"summary_control_Q1",index=False)
summary_control_decile.to_excel(writer,"summary_control_decile",index=False)
writer.save()
